In [1]:
proj_list = [
    'boringssl_total', 'c-ares_total',
    'freetype2_total', 'guetzli_total',
    'harfbuzz_total', 'libpng_total',
    'libssh_total', 'libxml2_total',
    'pcre_total', 'proj4_total',
    're2_total', 'sqlite3_total',
    'total', 'vorbis_total',
    'woff2_total', 'wpantund_total'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import data_loader as dl
import initializer as init
import trainer
import tester
import predictor
import model_util as mu
import pretrained_model as pm

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
prefix_np, postfix_np, label_np = data.getSingleProjectData(proj_list, proj_list[target_project])

Getting data for "boringssl_total" from "c-ares_total"
Getting data for "boringssl_total" from "freetype2_total"
Getting data for "boringssl_total" from "guetzli_total"
Getting data for "boringssl_total" from "harfbuzz_total"
Getting data for "boringssl_total" from "libpng_total"
Getting data for "boringssl_total" from "libssh_total"
Getting data for "boringssl_total" from "libxml2_total"
Getting data for "boringssl_total" from "pcre_total"
Getting data for "boringssl_total" from "proj4_total"
Getting data for "boringssl_total" from "re2_total"
Getting data for "boringssl_total" from "sqlite3_total"
Getting data for "boringssl_total" from "vorbis_total"
Getting data for "boringssl_total" from "woff2_total"
Getting data for "boringssl_total" from "wpantund_total"


In [6]:
test_prefix, test_postfix, test_label = data.getTestData(proj_list[target_project])

In [7]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

# train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
#     train_prefix, train_postfix, train_label, test_size = 0.1, random_state = 43
# )

In [8]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=500
    )

In [9]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/ctp2_tmp/tests')

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [11]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '2_tmp3'
epochs = 20

embed_dim = 128
max_len, source_code_tokens, token_choices = data.getInfo()
pretrained_token2vec = pm.load_pretrained_model(source_code_tokens, embed_dim)
pretrained_token2vec = torch.tensor(pretrained_token2vec)


input_size = max_len
hidden_size = 400
num_classes = max(token_choices) + 1
rnn_layers = 2

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.0

learning_rate = 0.001
weight_decay = 1e-4

model_name = "RNN"
optim_name = "Adam"
loss_fn_name = "CEL"

pretrained_model = pretrained_token2vec
freeze_embedding = False,

In [12]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=input_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
C_rnn(
  (emb): Embedding(213, 128)
  (lstm1): RNN(
    (rnn): LSTM(128, 400, num_layers=2, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(128, 400, num_layers=2, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=1600, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=213, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [13]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.482104   | 85.388325  | 0.284593 | 90.51  | 1150.04
   2    |   0.260014   | 91.277312  | 0.245594 | 91.59  | 1187.39
   3    |   0.233947   | 92.042482  | 0.227131 | 92.32  | 1193.42
   4    |   0.220826   | 92.469365  | 0.216545 | 92.68  | 1203.33
   5    |   0.212617   | 92.706843  | 0.217773 | 92.48  | 1194.19
   6    |   0.207484   | 92.882659  | 0.214230 | 92.64  | 1182.71
   7    |   0.202955   | 93.043218  | 0.212253 | 92.76  | 1183.32
   8    |   0.199979   | 93.135197  | 0.205169 | 93.02  | 1190.97
   9    |   0.197498   | 93.228452  | 0.198974 | 93.16  | 1184.30
  10    |   0.195680   | 93.263135  | 0.202266 | 93.02  | 1191.50
  11    |   0.194447   | 93.307579  | 0.204850 | 92.97  | 1181.00
  12    |   0.193263   | 93.351876  | 0.191204 | 93.43  | 1189.89
  13    |   0.191283   | 93.417513  | 0.1

In [14]:
mu.saveModel(title, model)

In [15]:
model = mu.getModel(title)
print(model)

C_rnn(
  (emb): Embedding(213, 128)
  (lstm1): RNN(
    (rnn): LSTM(128, 400, num_layers=2, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(128, 400, num_layers=2, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=1600, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=213, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [16]:
loss, acc = tester.test(test_dataloader=test_dataloader,
                        device=device,
                        model=model,
                        title=title)

test loss:  1.4578173361764009
test acc:  74.10902047592694


/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-p

saved precision and recall results to file!


/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arise/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
with open('../result/final2_tmp', 'a') as f:
    text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\n'
    f.write(text)

In [18]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!


In [19]:
# prefix =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# postfix = [2, 91, 2, 56, 2, 106, 47, 2, 134, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 91, 2, 119, 91, 2, 56, 2, 106, 47, 2, 119, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 91, 2, 119, 91, 2, 56, 2, 106, 47, 2, 119, 128, 50, 88, 33, 124, 49, 48, 134, 47, 2, 119]
# label_type = 128

In [20]:
# prefix =[0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 119, 2, 47, 134, 48, 49, 124, 33, 88, 50, 128, 119, 2, 47, 106, 2, 56, 2, 91, 119, 2, 91, 2, 56, 2, 48, 49, 2, 47, 48, 88, 50, 128, 119, 2, 47, 106, 2, 56, 2, 91, 119, 2, 91, 2, 56, 2, 48, 49, 2, 47, 48, 88, 50]
# postfix = [48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134, 128, 50, 88, 48, 47, 2, 49, 48, 2, 56, 2, 106, 91, 2, 56, 2, 91, 2, 56, 2, 106, 47, 2, 134]
# label_type = 128

In [21]:
# predictor.predict(prefix, postfix, model)